9:04 am 10th june, 2024:
In this notebook I'll be train a RNN on the word 'dogs' for quite some time such that when I supply it with the word 'dog' it should be able to predict the word 's'. I'm not quite sure if to train it on multiple words like 'dogs dune drunk dust damp dance ...' or just 'dogs dogs dogs dogs ...'. I think things will be clear as I move along. I've had my endeavors for past 3 days to understand the min-char-rnn.py by the OG himself, Mr. Karpathy. I hope to implement my own today.

In [1]:
import numpy as np

In [2]:
# let's first try to only iterate with the word dogs
data = 'dogs'
chars = list(dict.fromkeys(data))
size = len(data)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

I'll just keep about 5 hidden neurons 'arbitrarily' for now. I'm a bit confuesed about the seq_length's size. In Mr. Karpathy's code it's value was 25.
I'll try with 3 for now.

In [3]:
hidden_size = 6
seq_length = 3
learning_rate = 1e-1

In [4]:
Wxh = np.random.randn(hidden_size, size)*0.01
Whh = np.random.randn(hidden_size, hidden_size)*0.01
Why = np.random.randn(size, hidden_size)*0.01
bh = np.zeros((hidden_size, 1))
by = np.zeros((size, 1))

In [5]:
def lossFun(inputs, targets, hprev):
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  for t in range(len(inputs)):
    xs[t] = np.zeros((size,1))
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
    ys[t] = np.dot(Why, hs[t]) + by
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
    loss += -np.log(ps[t][targets[t],0])

  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])

  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext
    dhraw = (1 - hs[t] * hs[t]) * dh
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam)
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [6]:
def sample(h, seed_ix, n):
  x = np.zeros((size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(size), p=p.ravel())
    x = np.zeros((size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [7]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)
smooth_loss = -np.log(1.0/size)*seq_length
while smooth_loss > 0.001:
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1))
    p = 0
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  sample_ix = sample(hprev, inputs[0], 3)
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  # print('iter %d, loss: %f' % (n, smooth_loss))
  
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8)

  p += seq_length
  n += 1
    
text = 'd'
for i in range(3):
    ix = sample_ix[i]
    text += ix_to_char[ix]
print(text)

dogs


10:31am It's done. Did a while ago but writing it just now. Most of the works are simple copies from Mr. Karpathy's code. I just did a little "tinkering" with it to get what I wanted. So basically in the program what happens is we supply the training word 'dogs' to the model. Throughout the whole iterations, what is does is simply learn what word is the most likely one to come after a particular word. i.e. in the word 'dogs' if we supply the program with 'd' in the training phase then the program calculates the likelihood of getting 'o' or we train it such as way that it should be getting 'o'. If it doesn't there's error that's calculated which will be used to thus correct the corresponding weights and biases such that the output is always 'o' after we've inputted 'd'. It iterates in this way for as long as the error calculated isn't less than 0.001 in above case.

Finally when the error is below a particular level, we then print the letters following 'd' which will have been stored in the sample_ix variable. Thus, in this way we're able to predict 'dogs' from the word 'd' only by training the model in that way.